### H1B Data Exploration and Cleanup
______

#### Download Dataset

Run (MacOS/Linux)

```shell
chmod a+x download.sh
./download.sh
```
---------

#### Combine quarterly data into yearly data
Convert Excel to CSV since pandas read csv faster and this takes approximately one hour

In [2]:
import glob
import pandas as pd

In [10]:
years = ['2017', '2018', '2019', '2020', '2021', '2022']

print('Combining and Transforming excel file to csv')
for year in years:
    print(f'Starting Year {year}')
    file_list = glob.glob('data/' + year + '/*.xlsx')
    excel_list=[]
    num = len(file_list)
    for i in range(num):
        print(f'Reading File {i+1} / {num}')
        excel_list.append(pd.read_excel(file_list[i]))
    year_df = pd.DataFrame()
    for i in range(num):
        print(f'Combining File {i+1} / {num}')
        year_df = year_df.append(excel_list[i], ignore_index = True)
    print('Transforming File')
    year_df.to_csv ('data/' + year + '/raw.csv', index = None, header=True)
    print(f'File for Year {year} saved')

Combining and Transforming excel file to csv
Starting Year 2017
Reading File 1 / 1
Combining File 1 / 1


/var/folders/5v/ngr5t86d7t7flrv7l2wlwy4r0000gn/T/ipykernel_30671/2199148453.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  year_df = year_df.append(excel_list[i], ignore_index = True)


Transforming File
File for Year 2017 saved
Starting Year 2018
Reading File 1 / 1
Combining File 1 / 1


/var/folders/5v/ngr5t86d7t7flrv7l2wlwy4r0000gn/T/ipykernel_30671/2199148453.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  year_df = year_df.append(excel_list[i], ignore_index = True)


Transforming File
File for Year 2018 saved
Starting Year 2019
Reading File 1 / 1
Combining File 1 / 1


...

Combining File 3 / 4
Combining File 4 / 4
Transforming File
File for Year 2020 saved
Starting Year 2021
Reading File 1 / 4
Reading File 2 / 4
Reading File 3 / 4
Reading File 4 / 4
Combining File 1 / 4
Combining File 2 / 4


/var/folders/5v/ngr5t86d7t7flrv7l2wlwy4r0000gn/T/ipykernel_30671/2199148453.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  year_df = year_df.append(excel_list[i], ignore_index = True)


Combining File 3 / 4
Combining File 4 / 4
Transforming File
File for Year 2021 saved
Starting Year 2022
Reading File 1 / 4
Reading File 2 / 4
Reading File 3 / 4
Reading File 4 / 4
Combining File 1 / 4
Combining File 2 / 4


/var/folders/5v/ngr5t86d7t7flrv7l2wlwy4r0000gn/T/ipykernel_30671/2199148453.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  year_df = year_df.append(excel_list[i], ignore_index = True)


Combining File 3 / 4
Combining File 4 / 4
Transforming File
File for Year 2022 saved


Can safely delete Excel files (Optional)

Run (MacOS/Linux)
```
chmod a+x delete.sh
./delete.sh
```

______

#### Select Useful Columns

Take Year 2022 for example

In [3]:
df2022 = pd.read_csv('data/2022/raw.csv')
df2022.head()

/var/folders/5v/ngr5t86d7t7flrv7l2wlwy4r0000gn/T/ipykernel_33821/937220725.py:1: DtypeWarning: Columns (89) have mixed types. Specify dtype option on import or set low_memory=False.
  df2022 = pd.read_csv('data/2022/raw.csv')


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,WILLFUL_VIOLATOR,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL
0,I-200-21270-606997,Certified,2021-09-26,2021-10-01,NaN,H-1B,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,15-1132.00,"Software Developers, Applications",Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
1,I-200-21270-606867,Certified,2021-09-26,2021-10-01,NaN,H-1B,Designer,17-3013.00,Mechanical Drafters,Y,...,No,NaN,NaN,NaN,Disclose Business and Employment,Stacey,Francyne,NaN,Stacey Law Practice,francyne@staceylawpractice.com
2,I-200-21270-606846,Certified,2021-09-26,2021-10-01,NaN,H-1B,Data Analyst,15-2031.00,Operations Research Analysts,Y,...,No,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
3,I-200-21270-606842,Certified,2021-09-26,2021-10-01,NaN,H-1B,Pharmaceutical Chemist,19-2031.00,Chemists,Y,...,No,NaN,NaN,NaN,Disclose Business,JONNALAGADDA,SRINIVASA,R,"S. R. JONNALAGADDA, P.C.",Lawassociates@att.net
4,I-200-21270-606941,Certified,2021-09-26,2021-10-01,NaN,H-1B,Senior Systems Analyst JC60,15-1121.00,Computer Systems Analysts,Y,...,No,Yes,"$60,000 or higher annual wage",NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN


In [4]:
df2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626084 entries, 0 to 626083
Data columns (total 96 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   CASE_NUMBER                     626084 non-null  object 
 1   CASE_STATUS                     626084 non-null  object 
 2   RECEIVED_DATE                   626084 non-null  object 
 3   DECISION_DATE                   626084 non-null  object 
 4   ORIGINAL_CERT_DATE              35336 non-null   object 
 5   VISA_CLASS                      626084 non-null  object 
 6   JOB_TITLE                       626084 non-null  object 
 7   SOC_CODE                        626084 non-null  object 
 8   SOC_TITLE                       626084 non-null  object 
 9   FULL_TIME_POSITION              626084 non-null  object 
 10  BEGIN_DATE                      626084 non-null  object 
 11  END_DATE                        626084 non-null  object 
 12  TOTAL_WORKER_POS

Some of the columns are obviously useless

In [5]:
columns = [
    'JOB_TITLE',
    'SOC_TITLE',
    'FULL_TIME_POSITION',
    'BEGIN_DATE',
    'END_DATE',
    'EMPLOYER_NAME',
    'EMPLOYER_CITY',
    'EMPLOYER_STATE',
    'EMPLOYER_COUNTRY',
    'WAGE_RATE_OF_PAY_FROM',
    'WAGE_RATE_OF_PAY_TO',
    'WAGE_UNIT_OF_PAY',
    'PREVAILING_WAGE',
    'PW_UNIT_OF_PAY',
    'H_1B_DEPENDENT'
]

In [6]:
df2022 = df2022[columns]
df2022.head(5)

,JOB_TITLE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_COUNTRY,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,H_1B_DEPENDENT
0,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,"Software Developers, Applications",Y,2021-10-01,2024-09-30,SUEZ WATER MANAGEMENT & SERVICES INC.,Paramus,NJ,UNITED STATES OF AMERICA,92000.0,NaN,Year,87485.0,Year,No
1,Designer,Mechanical Drafters,Y,2021-10-11,2024-10-10,"Sensors, Inc",Saline,MI,UNITED STATES OF AMERICA,56181.0,61000.0,Year,56181.0,Year,No
2,Data Analyst,Operations Research Analysts,Y,2021-10-01,2024-09-30,"HSP Group, Inc.",Bonita Springs,FL,UNITED STATES OF AMERICA,60000.0,NaN,Year,59238.0,Year,No
3,Pharmaceutical Chemist,Chemists,Y,2021-10-01,2024-09-30,"ANIP Acquisitions, Inc.",Baudette,MN,UNITED STATES OF AMERICA,56160.0,NaN,Year,53851.0,Year,No
4,Senior Systems Analyst JC60,Computer Systems Analysts,Y,2021-10-15,2024-10-14,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,COLLEGE STATION,TX,UNITED STATES OF AMERICA,70325.0,NaN,Year,70325.0,Year,Yes


We also only want jobs in the US

In [7]:
df2022 = df2022[df2022['EMPLOYER_COUNTRY'] == 'UNITED STATES OF AMERICA']
df2022.head(5)

,JOB_TITLE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_COUNTRY,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,H_1B_DEPENDENT
0,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,"Software Developers, Applications",Y,2021-10-01,2024-09-30,SUEZ WATER MANAGEMENT & SERVICES INC.,Paramus,NJ,UNITED STATES OF AMERICA,92000.0,NaN,Year,87485.0,Year,No
1,Designer,Mechanical Drafters,Y,2021-10-11,2024-10-10,"Sensors, Inc",Saline,MI,UNITED STATES OF AMERICA,56181.0,61000.0,Year,56181.0,Year,No
2,Data Analyst,Operations Research Analysts,Y,2021-10-01,2024-09-30,"HSP Group, Inc.",Bonita Springs,FL,UNITED STATES OF AMERICA,60000.0,NaN,Year,59238.0,Year,No
3,Pharmaceutical Chemist,Chemists,Y,2021-10-01,2024-09-30,"ANIP Acquisitions, Inc.",Baudette,MN,UNITED STATES OF AMERICA,56160.0,NaN,Year,53851.0,Year,No
4,Senior Systems Analyst JC60,Computer Systems Analysts,Y,2021-10-15,2024-10-14,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,COLLEGE STATION,TX,UNITED STATES OF AMERICA,70325.0,NaN,Year,70325.0,Year,Yes


In [8]:
df2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625986 entries, 0 to 626083
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   JOB_TITLE              625986 non-null  object 
 1   SOC_TITLE              625986 non-null  object 
 2   FULL_TIME_POSITION     625986 non-null  object 
 3   BEGIN_DATE             625986 non-null  object 
 4   END_DATE               625986 non-null  object 
 5   EMPLOYER_NAME          625986 non-null  object 
 6   EMPLOYER_CITY          625986 non-null  object 
 7   EMPLOYER_STATE         625986 non-null  object 
 8   EMPLOYER_COUNTRY       625986 non-null  object 
 9   WAGE_RATE_OF_PAY_FROM  625986 non-null  float64
 10  WAGE_RATE_OF_PAY_TO    201930 non-null  float64
 11  WAGE_UNIT_OF_PAY       625986 non-null  object 
 12  PREVAILING_WAGE        625986 non-null  float64
 13  PW_UNIT_OF_PAY         625986 non-null  object 
 14  H_1B_DEPENDENT         611189 non-nu

In [9]:
df2022['FULL_TIME_POSITION'].describe()

count     625986
unique         2
top            Y
freq      617691
Name: FULL_TIME_POSITION, dtype: object

* END_DATE is three years after BEGIN_DATE. 

* WAGE_RATE_OF_PAY_TO has many NaN value.

* We do not need EMPLOYER_COUNTRY anymore.

* For H1-B, full-time position is mostly needed.

* So, we drop these columns.

In [10]:
df2022 = df2022.drop(labels=['END_DATE', 'WAGE_RATE_OF_PAY_TO', 'EMPLOYER_COUNTRY', 'FULL_TIME_POSITION'], axis=1)
df2022.head(5)

,JOB_TITLE,SOC_TITLE,BEGIN_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,H_1B_DEPENDENT
0,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,"Software Developers, Applications",2021-10-01,SUEZ WATER MANAGEMENT & SERVICES INC.,Paramus,NJ,92000.0,Year,87485.0,Year,No
1,Designer,Mechanical Drafters,2021-10-11,"Sensors, Inc",Saline,MI,56181.0,Year,56181.0,Year,No
2,Data Analyst,Operations Research Analysts,2021-10-01,"HSP Group, Inc.",Bonita Springs,FL,60000.0,Year,59238.0,Year,No
3,Pharmaceutical Chemist,Chemists,2021-10-01,"ANIP Acquisitions, Inc.",Baudette,MN,56160.0,Year,53851.0,Year,No
4,Senior Systems Analyst JC60,Computer Systems Analysts,2021-10-15,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,COLLEGE STATION,TX,70325.0,Year,70325.0,Year,Yes


______

#### Explore Data and Standarize Features

##### Date to Year

In [11]:
df2022['BEGIN_DATE'].describe()

count         625986
unique          1251
top       2022-10-01
freq          100201
Name: BEGIN_DATE, dtype: object

From consensus, salary among different days in one year should not differ from each other much.

So we care mostly about which year it is.

In [12]:
df2022['BEGIN_YEAR'] = df2022['BEGIN_DATE'].astype('str').map(lambda s: s[:4])
df2022['BEGIN_YEAR'].describe()

count     625986
unique         5
top         2022
freq      513283
Name: BEGIN_YEAR, dtype: object

In [13]:
df2022 = df2022.drop(labels=['BEGIN_DATE'], axis=1)

##### Standarize Salary

In [14]:
df2022['WAGE_UNIT_OF_PAY'].describe()

count     625986
unique         5
top         Year
freq      590810
Name: WAGE_UNIT_OF_PAY, dtype: object

In [15]:
df2022['PW_UNIT_OF_PAY'].describe()

count     625986
unique         5
top         Year
freq      591091
Name: PW_UNIT_OF_PAY, dtype: object

In [16]:
df2022['WAGE_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Bi-Weekly', 'Week', 'Month'], dtype=object)

* Different companies pay salary with different frequency. 

* Need to make them all in unit of Year.

* Assuming everyone works **40 hours per week, 4 weeks per month, 12 months per year**

In [17]:
for _, row in df2022.iterrows():
    unit = row['WAGE_UNIT_OF_PAY']
    wage = row['WAGE_RATE_OF_PAY_FROM']
    if unit == 'Hour':
        row['WAGE_RATE_OF_PAY_FROM'] = wage * 40 * 4 * 12
    elif unit == 'Bi-Weekly':
        row['WAGE_RATE_OF_PAY_FROM'] = wage * 2 * 12
    elif unit == 'Week':
        row['WAGE_RATE_OF_PAY_FROM'] = wage * 4 * 12
    elif unit == 'Month':
        row['WAGE_RATE_OF_PAY_FROM'] = wage * 12

    pw_unit = row['PW_UNIT_OF_PAY']
    pw = row['PREVAILING_WAGE']
    if pw_unit == 'Hour':
        row['PREVAILING_WAGE'] = pw * 40 * 4 * 12
    elif pw_unit == 'Bi-Weekly':
        row['PREVAILING_WAGE'] = pw * 2 * 12
    elif pw_unit == 'Week':
        row['PREVAILING_WAGE'] = pw * 4 * 12
    elif pw_unit == 'Month':
        row['PREVAILING_WAGE'] = pw * 12

In [18]:
df2022['SALARY'] = df2022['WAGE_RATE_OF_PAY_FROM'].round()
df2022['AVERAGE_SALARY'] = df2022['PREVAILING_WAGE'].round()
df2022 = df2022.drop(labels=['WAGE_UNIT_OF_PAY', 'PW_UNIT_OF_PAY', 'WAGE_RATE_OF_PAY_FROM', 'PREVAILING_WAGE'], axis=1)

In [19]:
df2022[['SALARY', 'AVERAGE_SALARY']].head(10)

,SALARY,AVERAGE_SALARY
0,92000.0,87485.0
1,56181.0,56181.0
2,60000.0,59238.0
3,56160.0,53851.0
4,70325.0,70325.0
5,90730.0,90730.0
6,130000.0,104291.0
7,90000.0,69888.0
8,98550.0,98550.0
9,106933.0,106933.0


##### Standarize City Name and Employer Name

In [20]:
df2022.head(5)

,JOB_TITLE,SOC_TITLE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,H_1B_DEPENDENT,BEGIN_YEAR,SALARY,AVERAGE_SALARY
0,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,"Software Developers, Applications",SUEZ WATER MANAGEMENT & SERVICES INC.,Paramus,NJ,No,2021,92000.0,87485.0
1,Designer,Mechanical Drafters,"Sensors, Inc",Saline,MI,No,2021,56181.0,56181.0
2,Data Analyst,Operations Research Analysts,"HSP Group, Inc.",Bonita Springs,FL,No,2021,60000.0,59238.0
3,Pharmaceutical Chemist,Chemists,"ANIP Acquisitions, Inc.",Baudette,MN,No,2021,56160.0,53851.0
4,Senior Systems Analyst JC60,Computer Systems Analysts,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,COLLEGE STATION,TX,Yes,2021,70325.0,70325.0


In [21]:
df2022['EMPLOYER_CITY'].str.upper().describe()

count       625986
unique        6253
top       NEW YORK
freq         31334
Name: EMPLOYER_CITY, dtype: object

In [22]:
df2022['EMPLOYER_CITY'].describe()

count      625986
unique       8363
top       Seattle
freq        26298
Name: EMPLOYER_CITY, dtype: object

In [23]:
df2022['EMPLOYER_NAME'].str.upper().describe()

count                      625986
unique                      66105
top       AMAZON.COM SERVICES LLC
freq                        16304
Name: EMPLOYER_NAME, dtype: object

In [24]:
df2022['EMPLOYER_NAME'].describe()

count                      625986
unique                      68960
top       Amazon.com Services LLC
freq                        14653
Name: EMPLOYER_NAME, dtype: object

* We can see that after making EMPLOYER_CITY and EMPLOYER_NAME Uppercase, their unique number changed.
  
* So we need to standarize them.
  
* Using **UPPERCASE** to standarize them

In [25]:
df2022['EMPLOYER'] = df2022['EMPLOYER_NAME'].str.upper()
df2022['CITY'] = df2022['EMPLOYER_CITY'].str.upper()
df2022 = df2022.drop(labels=['EMPLOYER_NAME', 'EMPLOYER_CITY'], axis=1)
df2022.head(5)

,JOB_TITLE,SOC_TITLE,EMPLOYER_STATE,H_1B_DEPENDENT,BEGIN_YEAR,SALARY,AVERAGE_SALARY,EMPLOYER,CITY
0,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,"Software Developers, Applications",NJ,No,2021,92000.0,87485.0,SUEZ WATER MANAGEMENT & SERVICES INC.,PARAMUS
1,Designer,Mechanical Drafters,MI,No,2021,56181.0,56181.0,"SENSORS, INC",SALINE
2,Data Analyst,Operations Research Analysts,FL,No,2021,60000.0,59238.0,"HSP GROUP, INC.",BONITA SPRINGS
3,Pharmaceutical Chemist,Chemists,MN,No,2021,56160.0,53851.0,"ANIP ACQUISITIONS, INC.",BAUDETTE
4,Senior Systems Analyst JC60,Computer Systems Analysts,TX,Yes,2021,70325.0,70325.0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,COLLEGE STATION


##### Standarize Job Titles

In [26]:
df2022['SOC_TITLE'].describe()

count                                625986
unique                                 1057
top       Software Developers, Applications
freq                                 167633
Name: SOC_TITLE, dtype: object

In [27]:
df2022['SOC_TITLE'].str.upper().describe()

count                                625986
unique                                  918
top       SOFTWARE DEVELOPERS, APPLICATIONS
freq                                 167786
Name: SOC_TITLE, dtype: object

In [28]:
df2022['JOB_TITLE'].describe()

count                625986
unique               139328
top       Software Engineer
freq                  31255
Name: JOB_TITLE, dtype: object

* How to distingush jobs with same SOC_TITLE?
  
* How to aggregate similar JOB_TITLEs?

In [29]:
df2022['JOB_TITLE'].str.upper().describe()

count                625986
unique               131836
top       SOFTWARE ENGINEER
freq                  36512
Name: JOB_TITLE, dtype: object